### Random Forest Regressor

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [3]:
df = pd.read_csv("merged_df.csv")
df.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,StateHoliday_encoded,isHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,StoreType_encoded,Assortment_encoded,PromoInterval_encoded
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01,4.516279e-02,8.213533e-01,5.422021e+03,7.336981e+00,2.008925e+03,5.005638e-01,1.164767e+01,1.007011e+03,1.207467e+00,9.351412e-01,3.093396e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01,2.836559e-01,3.830564e-01,7.706918e+03,2.674456e+00,4.999370e+00,4.999999e-01,1.532393e+01,1.005877e+03,1.365376e+00,9.938011e-01,1.421467e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01,1.000000e+00,1.900000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.100000e+02,6.000000e+00,2.008000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.325000e+03,8.000000e+00,2.010000e+03,1.000000e+00,1.000000e+00,2.009000e+03,0.000000e+00,0.000000e+00,1.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.880000e+03,9.000000e+00,2.011000e+03,1.000000e+00,2.200000e+01,2.012000e+03,3.000000e+00,2.000000e+00,1.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,7.586000e+04,1.200000e+01,2.015000e+03,1.000000e+00,5.000000e+01,2.015000e+03,3.000000e+00,2.000000e+00,3.000000e+00


In [4]:
# Convert date fields to datetime objects
df['Date'] = pd.to_datetime(df['Date'])
df['CompetitionOpenSinceDate'] = pd.to_datetime(df['CompetitionOpenSinceDate'])

# Calculate the duration of competition being open in months
df['CompetitionDurationMonths'] = (
    (df['Date'].dt.year - df['CompetitionOpenSinceDate'].dt.year) * 12 +
    (df['Date'].dt.month - df['CompetitionOpenSinceDate'].dt.month)
)

# Impute missing dates on Promo2Date with a placeholder value, e.g., '1900-01-01'
df['Promo2Date'].fillna('1900-01-01', inplace=True)

# Replace NaNs in 'CompetitionDurationMonths' with the median duration
df['CompetitionDurationMonths'].fillna(df['CompetitionDurationMonths'].median(), inplace=True)

# Drop the original date fields and other non-numeric or unnecessary columns
df = df.drop(['StoreType', 'Assortment', 'StateHoliday', 'PromoInterval', 'CompetitionOpenSinceMonth','CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear'], axis=1)

# --> impute median by store type within each year, tell why you used storetype. then once you have the months categorize them in ranges
# --> also remove the thing you did with competitionDate and promo2Date
# --> CompetitionDistanceCategory, show outliers graph

# Convert datetime columns to numeric features
df['Date_Year'] = df['Date'].dt.year
df['Date_Month'] = df['Date'].dt.month
df['Date_Day'] = df['Date'].dt.day

df['CompetitionOpenSinceDate_Year'] = df['CompetitionOpenSinceDate'].dt.year
df['CompetitionOpenSinceDate_Month'] = df['CompetitionOpenSinceDate'].dt.month
df['CompetitionOpenSinceDate_Day'] = df['CompetitionOpenSinceDate'].dt.day

df['Promo2Date'] = pd.to_datetime(df['Promo2Date'])
df['Promo2Date_Year'] = df['Promo2Date'].dt.year
df['Promo2Date_Month'] = df['Promo2Date'].dt.month
df['Promo2Date_Day'] = df['Promo2Date'].dt.day

# Drop the original datetime columns
df = df.drop(['Date', 'CompetitionOpenSinceDate', 'Promo2Date'], axis=1)

In [6]:
# Define the target variables and features
targets = ['Sales', 'Customers']
features = df.drop(targets, axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, df[targets], test_size=0.2, random_state=42)

In [10]:
# Initialize the RandomForestRegressor models
rf_sales = RandomForestRegressor(n_estimators=100, random_state=42)
rf_customers = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the RandomForestRegressor model for sales
rf_sales.fit(X_train, y_train['Sales'])

# Train the RandomForestRegressor model for customers
rf_customers.fit(X_train, y_train['Customers'])

RandomForestRegressor(random_state=42)

In [ ]:
# Predict on test data
sales_predictions = rf_sales.predict(X_test)
customers_predictions = rf_customers.predict(X_test)

In [ ]:
# Evaluate performance for Sales
mse_sales = mean_squared_error(y_test['Sales'], sales_predictions)
rmse_sales = np.sqrt(mse_sales)
rmspe_sales = rmse_sales / np.mean(y_test['Sales']) * 100

print(f'Mean Squared Error for Sales: {mse_sales}')
print(f'Root Mean Squared Error for Sales: {rmse_sales}')
print(f'Root Mean Squared Percentage Error for Sales: {rmspe_sales}')

Mean Squared Error for Sales: 681681.3903717938
Root Mean Squared Error for Sales: 825.6399883555748
Root Mean Squared Percentage Error for Sales: 14.329931932467122


In [ ]:
# Evaluate performance for Customers
mse_customers = mean_squared_error(y_test['Customers'], customers_predictions)
rmse_customers = np.sqrt(mse_customers)
rmspe_customers = rmse_customers / np.mean(y_test['Customers']) * 100

print(f'Mean Squared Error for Customers: {mse_customers}')
print(f'Root Mean Squared Error for Customers: {rmse_customers}')
print(f'Root Mean Squared Percentage Error for Customers: {rmspe_customers}')

Mean Squared Error for Customers: 5702.327715027378
Root Mean Squared Error for Customers: 75.51375844855941
Root Mean Squared Percentage Error for Customers: 11.947254980312639


In [ ]:
# Get feature importances
feature_importances = rf_sales.feature_importances_
feature_importances2 = rf_customers.feature_importances_

# Create a DataFrame to display feature importances
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df2 = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances2})
importance_df2 = importance_df2.sort_values(by='Importance', ascending=False)

# Print the top features
print("Top Features:")
print(importance_df.head())
print(importance_df2.head())

Top Features:
                         Feature  Importance
2                           Open    0.459925
7            CompetitionDistance    0.100048
0                          Store    0.081398
3                          Promo    0.073256
18  CompetitionOpenSinceDate_Day    0.044212
                         Feature  Importance
2                           Open    0.380239
7            CompetitionDistance    0.149041
0                          Store    0.102593
9              StoreType_encoded    0.054498
18  CompetitionOpenSinceDate_Day    0.045410
